### Example 4 - Comparison of two groups with continuous variables

### 2.1 What to do?
Next we will compare two groups of exemplary continuous data from the TotalSegmentator training dataset. Therefore load the exemplary data (ex4.csv) containing the age, gender and some volume and attenuation data from the segmented TotalSegmentator training dataset (same as in example 3). Now the goal is to compare two groups within the given data.

The function statsmed.comp_two_gr_continuous(), allows you to compare two input data with continuous variables. It requires as input the continuois data (x and y, that you want to compare) (please exclude NaN or None values), if the input data are independet (True) or dependent/related (False), the alternative {two-sided, less, greater}, the number of decimals it should return (if not given assumes 2), the mode and the number of decimals for the p value (if not given assumes 3).
The output depends on mode:
- if mode = = 'all' the function prints results of T-test for the means of two independent and dependent/related samples, Mann-Whitney U-Test of two independent samples and Wilcoxon signed-rank test of two dependent/related samples
- if mode = normal distribution - dependent on independent value the function prints results of T-test for the means of two independent or dependent/related samples
 - if mode = no normal distribution - dependent on independent value the function prints results of Mann-Whitney U-Test of two independent samples or Wilcoxon signed-rank test of two dependent/related samples
- if something else is given the respective output depends on whether the data is normal distributed or not normal distributed due to statsmed.stdnorm_test()
- the output for each line of the output: t-value rounded to number of given decimals; p-value rounded to number of decimals for significant p values;
- the given lines depend on the mode

If applied to your own data please be careful with possible NaN or None values in your data.

The exemplary data (same as in example 3) contains: sex, age, kidney_right_volume, kidney_left_volume, pancreas_volume, hip_left_intensity, hip_right_intensity, iliopsoas_left_volume, iliopsoas_right_volume, aorta_volume. Whereas Volumes are given in $mm^3$.

As already reported lot of volumes of the respective structure are zero, because the respective scan of the trainings dataset did not include this structure. Therefore we want to exclude these data. So let us use 10 ml as a lower cut-off. Moreover we want to report the volume in ml and $1 ml = 1000 mm^3$, so we need to divide the current value by 1000 (this has no effect on p-values).

So let us compare the volumes of the left and right kidney. The volume should be continuous.

In [2]:
import pandas
import numpy as np
from statsmed import statsmed
data = pandas.read_csv('ex4.csv',delimiter=',',on_bad_lines='skip')

print(data)

new_data = data[['kidney_right_volume','kidney_left_volume']][(data['kidney_right_volume'] > 10) & (data['kidney_left_volume'] > 10)]/1000
print(new_data)

statsmed.comp_two_gr_continuous(new_data['kidney_right_volume'],new_data['kidney_left_volume'],independent=False)


      Unnamed: 0 sex  age  kidney_right_volume  kidney_left_volume   
0              0   M   79           203708.250          184464.000  \
1              1   M   71           191288.250          181892.250   
2              2   M   66           125469.000          116474.625   
3              3   F   74           131415.750          118827.000   
4              4   M   85                0.000               0.000   
...          ...  ..  ...                  ...                 ...   
1085        1085   F   64                0.000               0.000   
1086        1086   M   81           171065.226          183714.724   
1087        1087   M   86           167487.750          186030.000   
1088        1088   M   65           192024.000          136856.250   
1089        1089   M   67                0.000               0.000   

      pancreas_volume  hip_left_intensity  hip_right_intensity   
0           72383.625               0.000                0.000  \
1          103372.875      

array([3.9904e+04, 1.0000e-03])

The left and right kidney are always in the same patient, thus the x- and y-data are dependent or related. Also, at least one of the x- or y-data is not normally distributed. Therefore a Wilcoxon signed-rank test is performed. As default we used "two-sided" for the hypothesis testing (less or greater would be the other possibilities).

We may now state: Within a patient there is a difference in the volume of the left and right kidney (p = 0.001). 

In [3]:
statsmed.get_desc(new_data['kidney_right_volume']-new_data['kidney_left_volume'])

The median with interquartile range (IQR) from the 25th to 75th percentile is: -3.72 (IQR: -14.89 - 9.61)
The pseudomedian with 95%-confidence interval from the signed-rank distribution is: -3.16 (CI: -5.05 - -1.27)


array([[ -3.72, -14.89,   9.61],
       [ -3.16,  -5.05,  -1.27]])

From this calculation we see that the left kidney appears to be larger than the right one from the segmentations.

Next let us test if there is a difference in the volume of the Aorta in patients with an age over 60 compared to patients with an age below or eqal 60. These data are not related, thus the x- and y-data are independent.

In [4]:
new_data = data[['age','aorta_volume']][data['aorta_volume'] >= 10]
new_data = new_data.reset_index()

statsmed.comp_two_gr_continuous(new_data['aorta_volume'][np.where(new_data['age'] > 60)[0]]/1000,new_data['aorta_volume'][np.where(new_data['age'] <= 60)[0]]/1000,independent=True,N_of_decimals = 0)


Testing normal distribution of x-data:
Shapiro-Wilk: No normal dsitribution (p-value = p < 0.001)
Kolmogorow-Smirnow: No normal dsitribution (p-value = p < 0.001)
At least one test indicates no normal distribution
Descriptive Statistic of the x-data with all returns:
The mean with standard deviation is: 264 ± 134
The mean with 95%-confidence interval is: 264 (CI: 248 - 281)
The median with interquartile range (IQR) from the 25th to 75th percentile is: 268 (IQR: 201 - 314)
The pseudomedian with 95%-confidence interval from the signed-rank distribution is: 261 (CI: 248 - 273)


Testing normal distribution of y-data:
Shapiro-Wilk: No normal dsitribution (p-value = p = 0.044)
Kolmogorow-Smirnow: Normal dsitribution (p-value = p = 0.49 
 	 - p-value >= 0.05 indicates no significant difference from normal distribution)
At least one test indicates no normal distribution
Descriptive Statistic of the y-data with all returns:
The mean with standard deviation is: 152 ± 79
The mean with 95%-confid

array([14416.,     0.])

We see that there is a significant difference with an p-value < 0.001 for two-tailed hypothesis testing. So, we may say that the volume of the Aorta with patients > 60 years 268 ml (IQR: 201 - 314 ml) is larger compared to patients with an age below or equal 60 years 166 ml (IQR: 107 - 192 ml; p < 0.001).

In case of a normal distribution of x- and y-data the function performes a t-test with independent or dependent/related samples respectively.

If the mode='all' is given, both test a t-test with independent samples and a Mann-Whitney U-Test is performed or a t-test with dependent/related samples and Wilcoxon signed-rank test dependent on wether the data is related or not, is performed.

In [5]:
statsmed.comp_two_gr_continuous(new_data['aorta_volume'][np.where(new_data['age'] > 60)[0]]/1000,new_data['aorta_volume'][np.where(new_data['age'] <= 60)[0]]/1000,independent=True,N_of_decimals = 0,mode='all')

Testing normal distribution of x-data:
Shapiro-Wilk: No normal dsitribution (p-value = p < 0.001)
Kolmogorow-Smirnow: No normal dsitribution (p-value = p < 0.001)
At least one test indicates no normal distribution
Descriptive Statistic of the x-data with all returns:
The mean with standard deviation is: 264 ± 134
The mean with 95%-confidence interval is: 264 (CI: 248 - 281)
The median with interquartile range (IQR) from the 25th to 75th percentile is: 268 (IQR: 201 - 314)
The pseudomedian with 95%-confidence interval from the signed-rank distribution is: 261 (CI: 248 - 273)


Testing normal distribution of y-data:
Shapiro-Wilk: No normal dsitribution (p-value = p = 0.044)
Kolmogorow-Smirnow: Normal dsitribution (p-value = p = 0.49 
 	 - p-value >= 0.05 indicates no significant difference from normal distribution)
At least one test indicates no normal distribution
Descriptive Statistic of the y-data with all returns:
The mean with standard deviation is: 152 ± 79
The mean with 95%-confid

array([[7.0000e+00, 0.0000e+00],
       [1.4416e+04, 0.0000e+00]])

### 2.2 What to write?

In the statistical analysis section of a manuscript you may write for unpaired groups:

"Comparison between unpaired groups was performed for continuous variables using a two-tailed Student’s t-test or a Mann-Whitney U-test depending on normality."

For paired groups you may write: "Comparison between paired groups was performed for continuous variables using a two-tailed t-test or a Wilcoxon signed-rank test depending on normality."

Additionally you may add: "Hypothesis testing was two-tailed. All P-values <0.05 were considered statistically significant."

In the results section of the manuscript you can now give the respective p-values.
